# 第一次测试成功模型

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import math

 
class ResidualBlock(nn.Module):
    
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(in_features, in_features, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(in_features, 0.8),
            nn.PReLU(),
            nn.Conv2d(in_features, in_features, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(in_features, 0.8),
        )
 
    def forward(self, x):
        return x + self.conv_block(x)
 
 
class Generator(nn.Module):
    def __init__(self, scale_factor=2, num_residual_blocks=16):
        super(Generator, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, stride=1, padding=4,padding_mode='reflect', bias=True)
        self.prelu = nn.PReLU()
        self.residual_blocks = nn.Sequential(*[ResidualBlock(64) for _ in range(num_residual_blocks)])
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=True)
        self.bn2 = nn.BatchNorm2d(64)
        self.upsample = nn.Sequential(
            nn.Conv2d(64, 256, kernel_size=3, stride=1, padding=1,padding_mode='reflect', bias=True),
            nn.PixelShuffle(scale_factor),
            nn.PReLU(),
            nn.Conv2d(64, 256, kernel_size=3, stride=1, padding=1,padding_mode='reflect', bias=True),
            nn.PixelShuffle(scale_factor),
            nn.PReLU(),
            nn.Conv2d(64, 3, kernel_size=9, stride=1, padding=4, bias=True)
        )
 
    def forward(self, x):
        out = self.conv1(x)
        out = self.prelu(out)
        residual = out
        out = self.residual_blocks(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        out = self.upsample(out)
        return out
 
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=True),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1, bias=True),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=1, bias=True),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1, bias=True),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),
            nn.Conv2d(512, 512, kernel_size=3, stride=2, padding=1, bias=True),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(512, 1024),
            nn.Dropout(),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        out = self.layer(x)
        return out

class TruncatedVGG19(nn.Module):
    def __init__(self, i=5, j=4):
        super(TruncatedVGG19, self).__init__()
        vgg19 = models.vgg19(pretrained=True).features
        self.features = nn.Sequential(*list(vgg19.children())[:((i-1)*5 + j)])
        for param in self.features.parameters():
            param.requires_grad = False

    def forward(self, x):
        return self.features(x)

# 模型2.0版本

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class ResidualBlock(nn.Module):
    def __init__(self, in_features, kernel_size=3):
        super(ResidualBlock, self).__init__()
        # 定义一个卷积块，包括两个卷积层，两个批归一化层和一个PReLU激活函数
        self.conv_block = nn.Sequential(
            nn.Conv2d(in_features, in_features, kernel_size=kernel_size, stride=1, padding=kernel_size//2),
            nn.BatchNorm2d(in_features),
            nn.PReLU(),
            nn.Conv2d(in_features, in_features, kernel_size=kernel_size, stride=1, padding=kernel_size//2),
            nn.BatchNorm2d(in_features),
        )

    def forward(self, x):
        # 前向传播过程中，将输入x和卷积块的输出相加
        return x + self.conv_block(x)

class Generator(nn.Module):
    def __init__(self, large_kernel_size=9, small_kernel_size=3, n_channels=64, n_blocks=16):
        super(Generator, self).__init__()
        # 定义生成器的各个组成部分，包括一个卷积层，一个PReLU激活函数，一个残差块，一个卷积层，一个批归一化层，以及一个上采样部分
        self.conv1 = nn.Conv2d(3, n_channels, kernel_size=large_kernel_size, stride=1, padding=large_kernel_size//2, padding_mode='reflect', bias=True)
        self.prelu = nn.PReLU()
        self.residual_blocks = nn.Sequential(*[ResidualBlock(n_channels, small_kernel_size) for _ in range(n_blocks)])
        self.conv2 = nn.Conv2d(n_channels, n_channels, kernel_size=small_kernel_size, stride=1, padding=small_kernel_size//2, bias=True)
        self.bn2 = nn.BatchNorm2d(n_channels)
        # 假设上采样因子为2，如果需要可以进行相应的修改
        self.upsample = nn.Sequential(
            nn.Conv2d(n_channels, n_channels * (2 ** 2), kernel_size=small_kernel_size, stride=1, padding=small_kernel_size//2, padding_mode='reflect', bias=True),
            nn.PixelShuffle(2),
            nn.PReLU(),
            nn.Conv2d(n_channels, n_channels * (2 ** 2), kernel_size=small_kernel_size, stride=1, padding=small_kernel_size//2, padding_mode='reflect', bias=True),
            nn.PixelShuffle(2),
            nn.PReLU(),
            nn.Conv2d(n_channels, 3, kernel_size=large_kernel_size, stride=1, padding=large_kernel_size//2, bias=True)
        )

    def forward(self, x):
        # 定义生成器的前向传播过程
        out = self.conv1(x)
        out = self.prelu(out)
        residual = out
        out = self.residual_blocks(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual  # 元素级别的相加
        out = self.upsample(out)
        return out

class Discriminator(nn.Module):
    def __init__(self, kernel_size=3, n_channels=64, n_blocks=8, fc_size=1024):
        super(Discriminator, self).__init__()
        layers = []
        # 假设在每个块后输入和输出通道数都翻倍
        for i in range(n_blocks):
            input_channels = n_channels if i == 0 else n_channels * (2 ** i)
            output_channels = n_channels * (2 ** (i + 1))
            layers.append(nn.Conv2d(input_channels, output_channels, kernel_size=kernel_size, stride=1 + i % 2, padding=kernel_size//2, bias=True))
            layers.append(nn.BatchNorm2d(output_channels))
            layers.append(nn.LeakyReLU(0.2))
            if i % 2 == 1:  # 每隔一个块进行下采样
                layers.append(nn.Conv2d(output_channels, output_channels, kernel_size=kernel_size, stride=2, padding=kernel_size//2, bias=True))
                layers.append(nn.BatchNorm2d(output_channels))
                layers.append(nn.LeakyReLU(0.2))

        self.layer = nn.Sequential(*layers)
        self.final = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(n_channels * (2 ** n_blocks), fc_size),
            nn.LeakyReLU(0.2),
            nn.Linear(fc_size, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # 定义判别器的前向传播过程
        out = self.layer(x)
        out = self.final(out)
        return out

# TruncatedVGG19
class TruncatedVGG19(nn.Module):
    def __init__(self, i=5, j=4):
        super(TruncatedVGG19, self).__init__()
        vgg19 = models.vgg19(pretrained=True).features
        self.features = nn.Sequential(*list(vgg19.children())[:((i-1)*5 + j)])
        for param in self.features.parameters():
            param.requires_grad = False

    def forward(self, x):
        return self.features(x)


# 加入SRRESNET

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import math

class ConvolutionalBlock(nn.Module):
    """
    ConvolutionalBlock类定义了一个卷积块，包括卷积层、批量归一化层（如果需要）和激活层。
    卷积层的参数（如输入/输出通道数、卷积核大小和步长）由构造函数参数提供。
    批量归一化层是可选的，由batch_norm参数决定是否添加。
    激活层的类型由activation参数决定，可以是PReLU、LeakyReLU、Tanh或者没有激活层。
    """

    def __init__(self, in_channels, out_channels, kernel_size, stride=1, batch_norm=False, activation=None):
        super(ConvolutionalBlock, self).__init__()

        if activation is not None:
            activation = activation.lower()
            assert activation in {'prelu', 'leakyrelu', 'tanh'}

        layers = list()

        layers.append(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride,
                      padding=kernel_size // 2))

        if batch_norm is True:
            layers.append(nn.BatchNorm2d(num_features=out_channels))

        if activation == 'prelu':
            layers.append(nn.PReLU())
        elif activation == 'leakyrelu':
            layers.append(nn.LeakyReLU(0.2))
        elif activation == 'tanh':
            layers.append(nn.Tanh())

        self.conv_block = nn.Sequential(*layers)

    def forward(self, input):
        """
        前向传播函数接受一个输入张量，并应用卷积块的层到输入张量上，然后返回输出张量。
        """
        output = self.conv_block(input)

        return output

class ResidualBlock(nn.Module):
    """
    一个残差块，包含两个卷积块并通过残差连接。
    """

    def __init__(self, kernel_size=3, n_channels=64):
        """
        :param kernel_size: 卷积核大小
        :param n_channels: 输入和输出通道数（相同，因为输入必须加到输出上）
        """
        super(ResidualBlock, self).__init__()

        # 第一个卷积块
        self.conv_block1 = ConvolutionalBlock(in_channels=n_channels, out_channels=n_channels, kernel_size=kernel_size,
                                              batch_norm=True, activation='PReLu')

        # 第二个卷积块
        self.conv_block2 = ConvolutionalBlock(in_channels=n_channels, out_channels=n_channels, kernel_size=kernel_size,
                                              batch_norm=True, activation=None)

    def forward(self, input):
        """
        前向传播。

        :param input: 输入图像，尺寸为 (N, n_channels, w, h) 的张量
        :return: 输出图像，尺寸为 (N, n_channels, w, h) 的张量
        """
        residual = input  # (N, n_channels, w, h)
        output = self.conv_block1(input)  # (N, n_channels, w, h)
        output = self.conv_block2(output)  # (N, n_channels, w, h)
        output = output + residual  # (N, n_channels, w, h)

        return output

class SubPixelConvolutionalBlock(nn.Module):
    """
    SubPixelConvolutionalBlock类定义了一个子像素卷积块，包括卷积层、像素重排层和PReLU激活层。
    卷积层的参数（如卷积核大小和输入/输出通道数）由构造函数参数提供。
    像素重排层用于上采样，上采样倍数由scaling_factor参数决定。
    """

    def __init__(self, kernel_size=3, n_channels=64, scaling_factor=2):
        super(SubPixelConvolutionalBlock, self).__init__()

        self.conv = nn.Conv2d(in_channels=n_channels, out_channels=n_channels * (scaling_factor ** 2),
                              kernel_size=kernel_size, padding=kernel_size // 2)
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor=scaling_factor)
        self.prelu = nn.PReLU()

    def forward(self, input):
        """
        前向传播函数接受一个输入张量，并应用子像素卷积块的层到输入张量上，然后返回输出张量。
        """
        output = self.conv(input)
        output = self.pixel_shuffle(output)
        output = self.prelu(output)

        return output

class SRResNet(nn.Module):
    def __init__(self, large_kernel_size=9, small_kernel_size=3, n_channels=64, n_blocks=16, scaling_factor=4):
        """
        初始化 SRResNet 模型的参数。

        :param large_kernel_size: 输入和输出卷积层的卷积核大小
        :param small_kernel_size: 中间卷积层的卷积核大小
        :param n_channels: 卷积层的通道数
        :param n_blocks: 残差块的数量
        :param scaling_factor: 图像上采样的因子，必须是2、4或8
        """
        super(SRResNet, self).__init__()

        # 第一个卷积块，使用大卷积核，不使用批量归一化，激活函数为PReLU
        self.conv_block1 = ConvolutionalBlock(in_channels=3, out_channels=n_channels, kernel_size=large_kernel_size,
                                              batch_norm=False, activation='PReLu')

        # 一系列残差块，每个块中包含一个跨块的跳跃连接
        self.residual_blocks = nn.Sequential(
            *[ResidualBlock(kernel_size=small_kernel_size, n_channels=n_channels) for _ in range(n_blocks)])

        # 第二个卷积块，使用小卷积核，使用批量归一化，不使用激活函数
        self.conv_block2 = ConvolutionalBlock(in_channels=n_channels, out_channels=n_channels,
                                              kernel_size=small_kernel_size,
                                              batch_norm=True, activation=None)

        # 一系列子像素卷积块，每个块的上采样因子为2，总的上采样因子等于scaling_factor
        n_subpixel_convolution_blocks = int(math.log2(scaling_factor))
        self.subpixel_convolutional_blocks = nn.Sequential(
            *[SubPixelConvolutionalBlock(kernel_size=small_kernel_size, n_channels=n_channels, scaling_factor=2) for _
              in range(n_subpixel_convolution_blocks)])

        # 最后一个卷积块，使用大卷积核，不使用批量归一化，激活函数为Tanh
        self.conv_block3 = ConvolutionalBlock(in_channels=n_channels, out_channels=3, kernel_size=large_kernel_size,
                                              batch_norm=False, activation='Tanh')

    def forward(self, lr_imgs):
        """
        定义 SRResNet 的前向传播过程。

        :param lr_imgs: 低分辨率输入图像，尺寸为 (N, 3, w, h) 的张量
        :return: 超分辨率输出图像，尺寸为 (N, 3, w * 缩放因子, h * 缩放因子) 的张量
        """
        output = self.conv_block1(lr_imgs)  # (N, 3, w, h)
        residual = output  # (N, n_channels, w, h)
        output = self.residual_blocks(output)  # (N, n_channels, w, h)
        output = self.conv_block2(output)  # (N, n_channels, w, h)
        output = output + residual  # (N, n_channels, w, h)
        output = self.subpixel_convolutional_blocks(output)  # (N, n_channels, w * 缩放因子, h * 缩放因子)
        sr_imgs = self.conv_block3(output)  # (N, 3, w * 缩放因子, h * 缩放因子)

        return sr_imgs
